In [11]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Activation, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3
from keras.optimizers import adam_v2

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
#     break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [12]:
train_dir = "/kaggle/input/csc4851-homework4/birds_400/train/"
validate_dir = "/kaggle/input/csc4851-homework4/birds_400/valid/"
test_dir = "/kaggle/input/csc4851-homework4/birds_400/test/"

train_img_gen = ImageDataGenerator(
        rescale=1 / 255.0,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True,
        fill_mode="nearest",
        validation_split=0.20)

test_img_gen = ImageDataGenerator(rescale=1 / 255.0)

In [13]:
batch_size = 10
train_gen = train_img_gen.flow_from_directory(
    directory=train_dir,
    target_size=(100, 100),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    subset='training',
    shuffle=True,
    seed=42
)
validate_gen = train_img_gen.flow_from_directory(
    directory=validate_dir,
    target_size=(100, 100),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    subset='validation',
    shuffle=True,
    seed=42
)
test_gen = test_img_gen.flow_from_directory(
    directory=test_dir,
    target_size=(100, 100),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

In [18]:
def prepare_model():    
    inception_model = InceptionV3(weights=None, include_top=False, input_shape=(100, 100, 3))

    x = inception_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    output_layer = Dense(400, activation='sigmoid')(x)

    cnn_model = Model(inputs=inception_model.input, outputs=output_layer)
    cnn_model.compile(adam_v2.Adam(learning_rate=0.0001), loss= 'categorical_crossentropy', metrics=['accuracy'])
    
    return cnn_model

In [ ]:
model = prepare_model()
model.fit_generator(train_gen,
                    validation_data = train_gen,
                    steps_per_epoch = train_gen.n//train_gen.batch_size,
                    validation_steps = validate_gen.n//validate_gen.batch_size,
                    epochs=5)

In [ ]:
validate_score = model.evaluate_generator(validate_gen)
print('Validate loss:', validate_score[0])
print('Validate accuracy:', validate_score[1])

In [ ]:
predict=model.predict_generator(test_gen)
y_classes = predict.argmax(axis=-1)
test_score = model.evaluate_generator(test_gen)
print('Test loss:', test_score[0])
print('Test accuracy:', test_score[1])